### Importing data sets
    will be using EMNIST data set imported from pytorch. 
    In the preprocessing stages
### TO-DO
    how to properly work with images (normally used to numbers)
    how to preprocess this data

In [3]:
import torchvision.transforms as transforms
import torchvision.datasets as EMNIST
import sklearn
import pandas as pd
import torch 
import torch.nn as nn
import numpy